In [1]:
import os
import re
import gc
import locale
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)

h:\Data Scientists Projects\Fine Tuning LLaMA-2 with Custom Dataset using PEFT and LoRA\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("timdettmers/openassistant-guanaco")
dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

In [3]:
dataset = dataset["train"].shuffle(seed=42).select(range(1000))
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [4]:
def transform_conversation(example):
    conversation_text = example["text"]
    segments = conversation_text.split("###")
    reformatted_segments = []

    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace("Human:", "").strip()

        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace("Assistant:", "").strip()
            reformatted_segments.append(f"<s>[INST] {human_text} [/INST] {assistant_text} </s>")
        else:
            reformatted_segments.append(f"<s>[INST] {human_text} [/INST] </s>")

    return {"text": "".join(reformatted_segments)}

In [5]:
transformed_dataset = dataset.map(transform_conversation)
transformed_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [6]:
transformed_dataset.push_to_hub("guanaco-llama2-1k")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.24s/ shards]
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/AndrOidBB7/guanaco-llama2-1k/commit/fbb40964cd6ffc6864f1d3fa5b1a0af865d5ede0', commit_message='Upload dataset', commit_description='', oid='fbb40964cd6ffc6864f1d3fa5b1a0af865d5ede0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/AndrOidBB7/guanaco-llama2-1k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='AndrOidBB7/guanaco-llama2-1k'), pr_revision=None, pr_num=None)

In [7]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "Llama-2-7b-chat-finetune"

In [8]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

In [9]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

In [10]:
output_dir = "./results"
num_train_epochs = 1

In [11]:
fp16 = False
bf16 = False

In [12]:
per_device_train_batch_size = 4
per_device_eval_batch_size = 4

In [13]:
gradient_accumulation_steps = 1
gradient_checkpointing = True

In [14]:
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001

In [15]:
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"

In [16]:
max_steps = -1
warmup_ratio = 0.03
group_by_length = True

In [17]:
save_steps = 0
logging_steps = 25

In [18]:
max_seq_length = None
packing = False
device_map = {"": 0}

In [19]:
dataset = load_dataset(dataset_name, split="train")
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [20]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
compute_dtype

torch.float16

In [21]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)
bnb_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [22]:
if torch.cuda.is_available() and compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training by setting bf16=True")
        print("=" * 80)
else:
    print("CUDA not available or conditions not met. Running on CPU...")

CUDA not available or conditions not met. Running on CPU...


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)
peft_config

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)
training_arguments

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)
trainer

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
logging.set_verbosity(logging.CRITICAL)

In [ ]:
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
prompt = "How to own a plane in the United States?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
del model
del pipe
del trainer

In [ ]:
gc.collect()
gc.collect()

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map
)
base_model

In [ ]:
model = PeftModel.from_pretrained(base_model, new_model)
model

In [ ]:
model = model.merge_and_unload()
model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
model.push_to_hub("entbappy/Llama-2-7b-chat-finetune", check_pr=True)
tokenizer.push_to_hub("entbappy/Llama-2-7b-chat-finetune",check_pr=True)